In [1]:
from sec_certs.dataset import CCDataset
from pathlib import Path
import shutil
import subprocess

In [2]:
# A directory with a dump of the documents that will enrich the dataset.
doc_dump_dir = Path("cc_certs_04_10_2022")
print(doc_dump_dir.exists())

# An output directory for the dataset.
dataset_dir = Path("cc_04_10_2022")
print(dataset_dir.exists())

True
False


In [3]:
# Print tool versions
subprocess.run(["pdftotext", "-v"])
subprocess.run(["tesseract", "-v"]);

tesseract 5.2.0
 leptonica-1.82.0
  libgif 5.2.1 : libjpeg 8d (libjpeg-turbo 2.1.3) : libpng 1.6.38 : libtiff 4.4.0 : zlib 1.2.12 : libwebp 1.2.4 : libopenjp2 2.5.0
 Found AVX2
 Found AVX
 Found FMA
 Found SSE4.1
 Found OpenMP 201511
 Found libarchive 3.6.1 zlib/1.2.12 liblzma/5.2.5 bz2lib/1.0.8 liblz4/1.9.3 libzstd/1.5.2
 Found libcurl/7.85.0 OpenSSL/1.1.1q zlib/1.2.12 brotli/1.0.9 zstd/1.5.2 libidn2/2.3.3 libpsl/0.21.1 (+libidn2/2.3.0) libssh2/1.10.0 nghttp2/1.50.0


pdftotext version 22.09.0
Copyright 2005-2022 The Poppler Developers - http://poppler.freedesktop.org
Copyright 1996-2011, 2022 Glyph & Cog, LLC


In [4]:
# Create the dataset
dset = CCDataset(root_dir=dataset_dir, name="cc", description="Final run on 04.10.2022")

In [5]:
# Get the metadata
dset.get_certs_from_web()

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:02<00:00,  1.00s/it]
The CSV cc_04_10_2022/web/cc_products_active.csv contains 8 duplicates by the primary key.
The CSV cc_04_10_2022/web/cc_products_archived.csv contains 10 duplicates by the primary key.
When merging certificates with dgst b26ce64e0c677e3d, the following mismatch occured: Attribute=security_level, self[security_level]={'ALC_DVS.1', 'EAL2+'}, other[security_level]={'EAL2'}
When merging certificates with dgst 7f043162f88a1c3a, the following mismatch occured: Attribute=not_valid_after, self[not_valid_after]=2024-09-19, other[not_valid_after]=2024-09-18
When merging certificates with dgst 109eb2158ca6a2f9, the following mismatch occured: Attribute=security_level, self[security_level]={'EAL4+', 'AVA_MSU.3', 'ATE_DPT.2', 'ADV_IMP.2', 'AVA_VLA.4'}, other[security_level]={'ADV_DVS.2', 'EAL5+', 'AVA_MSU.3', 'AVA_VLA.4'}
When merging certifi

In [8]:
# Process the profiles?
dset.process_protection_profiles()

Duplicate entry in PP dataset: ('Protection Profile for Enterprise Security Management Policy Management', 'https://www.commoncriteriaportal.org/files/ppfiles/PP_ESM_PM_V1.4.pdf')
Duplicate entry in PP dataset: ('Stateful Traffic Filter Firewall Extended Package for Network Device Protection Profile', 'https://www.commoncriteriaportal.org/files/ppfiles/PP_ND_TFFWEP_V1.0.pdf')
Duplicate entry in PP dataset: ('Stateful Traffic Filter Firewall Extended Package for Network Device Protection Profile', 'https://www.commoncriteriaportal.org/files/ppfiles/PP_ND_TFFWEP_V1.0.pdf')
Duplicate entry in PP dataset: ('Public Key-Enabled Application Family of Protection Profiles', 'https://www.commoncriteriaportal.org/files/ppfiles/PP_VID3004-PP.pdf')
Duplicate entry in PP dataset: ('Protection Profile for Wireless Local Area Network (WLAN) Access Systems', 'https://www.commoncriteriaportal.org/files/ppfiles/PP_WLAN_AS_V1.0.pdf')
Duplicate entry in PP dataset: ('PC Client Specific Trusted Platform Mod

In [9]:
# Enrich the dataset with the pre-downloaded PDFs (collected over multiple runs to fill in the timeouts and disappearances).
dset.reports_pdf_dir.mkdir(parents=True, exist_ok=True)
dset.targets_pdf_dir.mkdir(parents=True, exist_ok=True)
for cert in dset:
    report_pdf = doc_dump_dir / "report" / "pdf" / f"{cert.dgst}.pdf"
    if report_pdf.exists():
        shutil.copy(report_pdf, cert.state.report_pdf_path)
        cert.state.report_download_ok = True
    target_pdf = doc_dump_dir / "target" / "pdf" / f"{cert.dgst}.pdf"
    if target_pdf.exists():
        shutil.copy(target_pdf, cert.state.st_pdf_path)
        cert.state.st_download_ok = True

In [10]:
# Try to download last missing (or ones not present in the pre-downloaded)
dset.download_all_pdfs(fresh=False)

Cert dgst: e27eb9efbd67c4f7 failed to download report from https://www.commoncriteriaportal.org/files/epfiles/Document%20SAFER%20Blue%202%20Security%20Target%20V1.10_???.pdf, code: nok
Cert dgst: d79ecebfe21f2144 failed to download report from https://www.commoncriteriaportal.org/files/epfiles/st_vid10024a-vr.pdf, code: nok
Cert dgst: 8ac22970bd8042d2 failed to download report from https://www.commoncriteriaportal.org/files/epfiles/st_vid4016b-vr.pdf, code: nok
Cert dgst: 02482228eb547c15 failed to download report from https://www.commoncriteriaportal.org/files/epfiles/st_vid4016a-vr.pdf, code: nok
Cert dgst: a90d78924e91dc6c failed to download report from https://www.commoncriteriaportal.org/files/epfiles/ANSSI-CC_2010-03en1.pdf, code: nok
Cert dgst: ace7ea6d7f58dbb1 failed to download report from https://www.commoncriteriaportal.org/files/epfiles/0308a_pdf.pdf, code: nok
Cert dgst: 811ca609aebe2e96 failed to download report from https://www.commoncriteriaportal.org/files/epfiles/st_v

In [11]:
# Convert all
dset.convert_all_pdfs()

Converting reports to txt:   2%|█▋                                                                                          | 96/5115 [00:07<06:29, 12.89it/s]Error when converting pdf->txt: poppler error creating document
Detected garbage during conversion of cc_04_10_2022/certs/reports/pdf/bfc86f14e2484335.pdf
Converting reports to txt:   2%|██                                                                                         | 114/5115 [00:09<06:14, 13.36it/s]Error during OCR of cc_04_10_2022/certs/reports/pdf/bfc86f14e2484335.pdf, using garbage: pdftoppm failed: 1
Cert dgst: bfc86f14e2484335 failed to convert report pdf->txt
Converting reports to txt:   4%|███▋                                                                                       | 204/5115 [00:14<05:30, 14.87it/s]Detected garbage during conversion of cc_04_10_2022/certs/reports/pdf/20874d4fa6e4c878.pdf
Detected garbage during conversion of cc_04_10_2022/certs/reports/pdf/3477723044183b31.pdf
Detected garbage du

Converting reports to txt:  10%|████████▌                                                                               | 497/5115 [13:38<15:10:43, 11.83s/it]Detected garbage during conversion of cc_04_10_2022/certs/reports/pdf/6f8d7a6a1dea6a3a.pdf
Detected garbage during conversion of cc_04_10_2022/certs/reports/pdf/e6cffd14e732a030.pdf
Converting reports to txt:  17%|███████████████                                                                            | 850/5115 [18:10<09:48,  7.24it/s]Error when converting pdf->txt: poppler error creating document
Error when converting pdf->txt: poppler error creating document
Detected garbage during conversion of cc_04_10_2022/certs/reports/pdf/4de4a9f436958574.pdf
Error when converting pdf->txt: poppler error creating document
Detected garbage during conversion of cc_04_10_2022/certs/reports/pdf/02958808b57334f0.pdf
Detected garbage during conversion of cc_04_10_2022/certs/reports/pdf/98668cafb8dda26f.pdf
Error during OCR of cc_04_10_2022/cer

Detected garbage during conversion of cc_04_10_2022/certs/reports/pdf/6db03eff148934a4.pdf
Error when converting pdf->txt: poppler error creating document
Detected garbage during conversion of cc_04_10_2022/certs/reports/pdf/f9c8da9deff77ab5.pdf
Error during OCR of cc_04_10_2022/certs/reports/pdf/f9c8da9deff77ab5.pdf, using garbage: pdftoppm failed: 1
Cert dgst: f9c8da9deff77ab5 failed to convert report pdf->txt
Converting reports to txt:  87%|██████████████████████████████████████████████████████████████████████████████            | 4433/5115 [24:32<00:58, 11.59it/s]Error when converting pdf->txt: poppler error creating document
Detected garbage during conversion of cc_04_10_2022/certs/reports/pdf/7147c2f70d983d57.pdf
Converting reports to txt:  87%|██████████████████████████████████████████████████████████████████████████████▏           | 4447/5115 [24:33<00:54, 12.18it/s]Error during OCR of cc_04_10_2022/certs/reports/pdf/7147c2f70d983d57.pdf, using garbage: pdftoppm failed: 1
Cert 

Detected garbage during conversion of cc_04_10_2022/certs/targets/pdf/ce987167d42db722.pdf
Converting targets to txt:  68%|█████████████████████████████████████████████████████████████▌                            | 3445/5032 [27:21<06:55,  3.82it/s]Detected garbage during conversion of cc_04_10_2022/certs/targets/pdf/f65cab49ea5422dd.pdf
Detected garbage during conversion of cc_04_10_2022/certs/targets/pdf/731102b849e8d104.pdf
Converting targets to txt:  77%|████████████████████████████████████████████████████████████████████▉                     | 3856/5032 [29:22<04:34,  4.28it/s]Detected garbage during conversion of cc_04_10_2022/certs/targets/pdf/bc31fd0273ab72f8.pdf
Detected garbage during conversion of cc_04_10_2022/certs/targets/pdf/18e5179ba124af1d.pdf
Converting targets to txt:  79%|███████████████████████████████████████████████████████████████████████                   | 3974/5032 [29:59<06:06,  2.89it/s]Detected garbage during conversion of cc_04_10_2022/certs/targets/pdf/2

Converting targets to txt:   0%|                                                                                                        | 0/5 [00:00<?, ?it/s]Error when converting pdf->txt: poppler error creating document
Detected garbage during conversion of cc_04_10_2022/certs/targets/pdf/7147c2f70d983d57.pdf
Error when converting pdf->txt: poppler error creating document
Error when converting pdf->txt: poppler error creating document
Error when converting pdf->txt: poppler error creating document
Error when converting pdf->txt: poppler error creating document
Error during OCR of cc_04_10_2022/certs/targets/pdf/7147c2f70d983d57.pdf, using garbage: pdftoppm failed: 1
Detected garbage during conversion of cc_04_10_2022/certs/targets/pdf/a2b962c7f1d2bc56.pdf
Detected garbage during conversion of cc_04_10_2022/certs/targets/pdf/bfc86f14e2484335.pdf
Detected garbage during conversion of cc_04_10_2022/certs/targets/pdf/422dc5758723c7d1.pdf
Detected garbage during conversion of cc_04_10_202

In [12]:
# Analyze all
dset.analyze_certificates()

Extracting report metadata:   0%|                                                                                                    | 0/5107 [00:00<?, ?it/s]Failed to read metadata of certs/reports/pdf/1e91595cd5e7fece.pdf, error: Unexpected escaped string: b'\x83'
Failed to read metadata of certs/reports/pdf/451e0806dfcfd29d.pdf, error: PDF metadata unavailable
Extracting report metadata:  35%|██████████████████████████████▊                                                         | 1786/5107 [00:15<00:22, 146.12it/s]Failed to read metadata of certs/reports/pdf/a62c55533f4e1636.pdf, error: PDF metadata unavailable
Failed to read metadata of certs/reports/pdf/d3568613c552f9e8.pdf, error: EOF marker not found
Extracting report metadata:  45%|████████████████████████████████████████                                                | 2323/5107 [00:18<00:16, 165.54it/s]Failed to read metadata of certs/reports/pdf/1c5deb472a197eb6.pdf, error: PDF metadata unavailable
Failed to read metadata o

Failed to read metadata of certs/reports/pdf/518b3066ebfabcd3.pdf, error: PDF metadata unavailable
Extracting target metadata:   1%|▊                                                                                          | 45/5027 [00:01<01:58, 42.14it/s]PdfReadWarning: Multiple definitions in dictionary at byte 0x724b1 for key /Im312 [generic.py:590]
Failed to read metadata of certs/targets/pdf/d47a03be1cb342b1.pdf, error: PDF metadata unavailable
Extracting target metadata:   2%|██▏                                                                                       | 120/5027 [00:02<01:23, 58.92it/s]Failed to read metadata of certs/targets/pdf/5af5a1b535422b51.pdf, error: PDF metadata unavailable
Failed to read metadata of certs/targets/pdf/21d6eff0bfe3971f.pdf, error: PDF metadata unavailable
Extracting target metadata:   7%|█████▉                                                                                    | 331/5027 [00:11<04:10, 18.73it/s]Failed to read metadata of cert

Failed to read metadata of certs/targets/pdf/bbcfe0aee7ea78ea.pdf, error: PDF metadata unavailable
Failed to read metadata of certs/targets/pdf/f0bd6a29eee94a2c.pdf, error: PDF metadata unavailable
Failed to read metadata of certs/targets/pdf/463ecd64b7506048.pdf, error: PDF metadata unavailable
Predicting CPE matches with the classifier: 100%|█████████████████████████████████████████████████████████████████████████| 5129/5129 [03:20<00:00, 25.64it/s]


parsing cpe matching (by NIST) dictionary: 100%|███████████████████████████████████████████████████████████████████| 398394/398394 [00:27<00:00, 14339.71it/s]
Building-up lookup dictionaries for fast CVE matching: 100%|███████████████████████████████████████████████████████| 196907/196907 [00:12<00:00, 16078.76it/s]
Computing related CVES: 100%|████████████████████████████████████████████████████████████████████████████████████████████| 698/698 [00:00<00:00, 82482.23it/s]


In [13]:
# And do maintenance updates as well
dset.process_maintenance_updates()

Converting reports to txt:  91%|███████████████████████████████████████████████████████████████████████████████████▎        | 414/457 [00:11<00:01, 38.81it/s]ERROR:sec_certs.utils.pdf:Error when converting pdf->txt: poppler error creating document
ERROR:sec_certs.utils.pdf:Error during OCR of cc_04_10_2022/certs/maintenances/reports/pdf/cert_8ba5d4d02ea73d3d_update_d879046e9be61396.pdf, using garbage: pdftoppm failed: 1
ERROR:sec_certs.sample.certificate:Cert dgst: cert_8ba5d4d02ea73d3d_update_d879046e9be61396 failed to convert report pdf->txt
Converting targets to txt: 100%|████████████████████████████████████████████████████████████████████████████████████████████| 455/455 [05:48<00:00,  1.31it/s]
ERROR:sec_certs.utils.pdf:Error when converting pdf->txt: poppler error creating document
Converting reports to txt:   0%|                                                                                                        | 0/1 [00:00<?, ?it/s]WARNING:sec_certs.utils.pdf:Detected garbag

In [14]:
# Finally, dump it all
dset.to_json()